In [44]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

In [2]:
import urllib.parse
from bs4 import BeautifulSoup

In [3]:
import requests
import brotli

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [5]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [6]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

In [48]:
logfile_name = "_seleniumtest_log.txt"

# import sk & tw politician test data

In [7]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [8]:
sktest.shape

(10114, 24)

In [9]:
# sktest["URL_NamuWiki"]=np.nan

In [10]:
sktest[100:200]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
100,강선광,姜善光,19610129,남,53,구·시·군의회의원선거,20140604,Legislator,충청남도,천안시서북구,...,등록,0,NaN,NaN,NaN,정당인,호서대학교 경영학과 졸업,(전)쌍용1동 주민자치위원장,(현)충청남도바르게살기협의회 부회장,NaN
101,강선구,姜先求,19620813,남,51,구·시·군의 장선거,20140604,Chief,인천광역시,중구,...,등록,0,NaN,NaN,NaN,정당인,경희대학교 국어국문학과 졸업,(전)민주당 중구·동구·옹진군 지역위원장,(전)국회정책연구위원,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
102,강선구,姜善求,19490218,남,65,구·시·군의회의원선거,20140604,Legislator,대전광역시,서구,...,등록,0,NaN,NaN,NaN,무직,미기재,(전)둔산상포장의사대표,(전)월평지역의용소방대장,https://namu.wiki/w/%EA%B0%95%EC%84%A0%EA%B5%AC
103,강선우,姜仙祐,19780602,여,37,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,등록,0,NaN,NaN,NaN,교수,미국 위스콘신대 가족학 박사(6년),(현)사우스다코타주립대 교수,(현)전미 노인학회 정회원,NaN
104,강선화,姜善花,19800818,여,33,시·도의회의원선거,20140604,Legislator,서울특별시,영등포구,...,등록,0,NaN,NaN,NaN,정당인,전북대학교 심리학과 졸업,(전)민주노총 금속노조 선전부장,(현)영등포 뉴타운폐지와 마을공동체 추진포럼 운영위원,NaN
105,강성균,姜性均,19520223,남,62,교육의원선거,20140604,Education Council Member,제주특별자치도,제주시,...,등록,1,44340.0,57.37,NaN,교육활동,제주대학교 교육대학원(석사) 교육학과 국어교육전공 취득,(전) 제주과학고등학교 교장,(전) 탐라교육원장,NaN
106,강성길,姜成吉,19630823,남,50,구·시·군의회의원선거,20140604,Legislator,서울특별시,서초구,...,등록,0,NaN,NaN,NaN,제6대 서초구의원,중앙대학교 사회개발 대학원 사회복지학과 석사과정 재학,(현)제6대 서초구의회 행정복지위원장,(현)세금바로쓰기 납세자 운동 서울시지부 회장,NaN
107,강성길,姜誠吉,19570808,남,56,구·시·군의회의원선거,20140604,Legislator,대구광역시,동구,...,등록,0,NaN,NaN,NaN,직장인,중앙대학교 사회개발대학원 보건행정학과 석사 수료,(전)새정치민주연합 중앙당 정책당원,(현)민주노총 및 공공운수노조 대의원,NaN
108,강성민,姜成旼,19710806,남,42,시·도의회의원선거,20140604,Legislator,제주특별자치도,제주시,...,등록,0,NaN,NaN,NaN,정치인,제주대학교 법정대학 행정학과 졸업,(현)제주미래비전연구원 제주생활정책포럼 대표,(전)제주특별자치도의회 정책자문위원,https://namu.wiki/w/%EA%B0%95%EC%84%B1%EB%AF%BC
109,강성봉,姜聖峰,19710201,남,43,광역의원비례대표선거,20140604,Party List Legislator,전라북도,NaN,...,등록,0,NaN,NaN,NaN,회사원,고려대학교 행정대학원 졸업,(현)새정치민주연합전국청년위원회부위원장,NaN,NaN


In [72]:
# percent duplicates

1 - (sktest["Name_Native"].drop_duplicates().shape[0] / sktest.shape[0])

0.1740944433908591

# scrape namuwiki with selenium

In [57]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

In [98]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""

        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                #sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,url,"\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")

            else:

                # update HasNamuWiki content field in dataframe
                # sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                # save result to log file
                log_output = ",".join([str(index),p.Name_Native,"False","\n"])
                fout = open(fileout_path+logfile_name,"a",encoding="utf-8")
                fout.write(log_output)
                fout.close()

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
            
        except KeyboardInterrupt:
        
            print("Keyboard Interrupt")
            
            break

        except Exception as e:

            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            print(e)





61 강민국 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD Number of <p> elements: 4 Has Content
109 강선우 https://namu.wiki/w/%EA%B0%95%EC%84%A0%EC%9A%B0 Number of <p> elements: 4 Error
Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00676463+2188387]
	Ordinal0 [0x0060E461+1762401]
	Ordinal0 [0x00523D78+802168]
	Ordinal0 [0x00517210+750096]
	Ordinal0 [0x0051675A+747354]
	Ordinal0 [0x00515D3F+744767]
	Ordinal0 [0x00514C28+740392]
	Ordinal0 [0x00515228+741928]
	Ordinal0 [0x0051EF2F+782127]
	Ordinal0 [0x00529FBB+827323]
	Ordinal0 [0x0052D310+840464]
	Ordinal0 [0x005154F6+742646]
	Ordinal0 [0x00529BF3+826355]
	Ordinal0 [0x0057CF6D+1167213]
	Ordinal0 [0x0056C5F6+1099254]
	Ordinal0 [0x00546BE0+945120]
	Ordinal0 [0x00547AD6+948950]
	GetHandleVerifier [0x009171F2+2712546]
	GetHandleVerifier [0x0090886D+2652765]
	GetHandleVerifier [0x0070002A+520730]
	GetHandleVerifier

2267 김용재 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%9E%AC Number of <p> elements: 4 Has Content
2268 김용재 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%9E%AC Number of <p> elements: 4 Has Content
2269 김용주 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A3%BC Number of <p> elements: 4 Has Content
2270 김용진 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84 Number of <p> elements: 4 Has Content
2271 김용진 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84 Number of <p> elements: 4 Has Content
2272 김용집 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%91 Number of <p> elements: 4 Has Content
2273 김용집 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%91 Number of <p> elements: 4 Has Content
2274 김용찬 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%B0%AC Number of <p> elements: 4 Has Content
2275 김용철 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%B2%A0 Number of <p> elements: 4 Has Content
2276 김용철 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%B2%A0 Number of <p> elements: 4 Has Content
2277 김용철 https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC

2357 김윤정 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A0%95 Number of <p> elements: 4 Has Content
2358 김윤정 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A0%95 Number of <p> elements: 4 Has Content
2359 김윤주 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A3%BC Number of <p> elements: 4 Has Content
2360 김윤진 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A7%84 Number of <p> elements: 4 Has Content
2361 김윤진 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A7%84 Number of <p> elements: 4 Has Content
2362 김윤진 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A7%84 Number of <p> elements: 4 Has Content
2363 김윤진 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%A7%84 Number of <p> elements: 4 Has Content
2364 김윤철 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%B2%A0 Number of <p> elements: 4 Has Content
2365 김윤철 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%EC%B2%A0 Number of <p> elements: 4 Has Content
2366 김윤태 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%ED%83%9C Number of <p> elements: 4 Has Content
2367 김윤태 https://namu.wiki/w/%EA%B9%80%EC%9C%A4%ED

2424 김인겸 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EA%B2%B8 Number of <p> elements: 4 Has Content
2425 김인곤 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EA%B3%A4 Number of <p> elements: 4 Has Content
2426 김인규 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EA%B7%9C Number of <p> elements: 4 Has Content
2427 김인기 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EA%B8%B0 Number of <p> elements: 4 Has Content
2428 김인기 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EA%B8%B0 Number of <p> elements: 4 Has Content
2429 김인덕 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EB%8D%95 Number of <p> elements: 4 Has Content
2430 김인두 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EB%91%90 Number of <p> elements: 5 Empty
2431 김인련 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EB%A0%A8 Number of <p> elements: 5 Empty
2432 김인배 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EB%B0%B0 Number of <p> elements: 4 Has Content
2433 김인서 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%84%9C Number of <p> elements: 5 Empty
2434 김인선 https://namu.wiki/w/%EA%B9%80%EC%9D%B8%EC%84%A0 Number of <

2496 김재동 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EB%8F%99 Number of <p> elements: 4 Has Content
2497 김재무 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EB%AC%B4 Number of <p> elements: 4 Has Content
2498 김재민 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EB%AF%BC Number of <p> elements: 4 Has Content
2499 김재범 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EB%B2%94 Number of <p> elements: 4 Has Content
2500 김재복 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EB%B3%B5 Number of <p> elements: 5 Empty
2501 김재상 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%83%81 Number of <p> elements: 4 Has Content
2502 김재선 https://namu.wiki/w/%EA%B9%80%EC%9E%AC%EC%84%A0 Number of <p> elements: 4 Error
Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00676463+2188387]
	Ordinal0 [0x0060E461+1762401]
	Ordinal0 [0x00523D78+802168]
	Ordinal0 [0x00517210+750096]
	Ordinal0 [0x0051675A+747354]
	Ordinal0 [0x00515D3F+744767

2575 김정식 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%8B%9D Number of <p> elements: 4 Has Content
2576 김정심 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%8B%AC Number of <p> elements: 5 Empty
2577 김정애 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%95%A0 Number of <p> elements: 4 Has Content
2578 김정애 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%95%A0 Number of <p> elements: 4 Has Content
2579 김정언 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%96%B8 Number of <p> elements: 5 Empty
2580 김정연 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%97%B0 Number of <p> elements: 4 Has Content
2581 김정열 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%97%B4 Number of <p> elements: 4 Has Content
2582 김정열 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%97%B4 Number of <p> elements: 4 Has Content
2583 김정영 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%98%81 Number of <p> elements: 4 Has Content
2584 김정오 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%98%A4 Number of <p> elements: 5 Empty
2585 김정용 https://namu.wiki/w/%EA%B9%80%EC%A0%95%EC%9A%A9 Number of <

In [ ]:
# things to check

# 0. drop duplicates. some duplicate pages are different (at least size differs). why?  

# 0.1 drop duplicates (by=[Name_Native, Name_Chinese, Birthdate]) in TW dataset, before scraping

# 1. recorded links are valid (no false positives); non-recorded links are actually empty (no false negatives)

# 2. disambiguation? what about names that link to more than one person?
#
#       - disambiguation links are usually included in the page
#
#       - example
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#               정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#               https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)
#
#       - another example
#               https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EC%A7%84

# update sktest with selenium log from latest run

In [99]:
log_columns = ["index","Name_Native","url","dropme"]
log = pd.read_csv(fileout_path+logfile_name,header=None,encoding="utf-8",names=log_columns)
log.drop(columns=["dropme"],inplace=True)
log

,index,Name_Native,url
0,61,강민국,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD
1,711,김경열,https://namu.wiki/w/%EA%B9%80%EA%B2%BD%EC%97%B4
2,2230,김용갑,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B0%91
3,2231,김용경,False
4,2232,김용권,False
5,2233,김용규,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
6,2234,김용규,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
7,2235,김용균,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0
8,2236,김용균,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0
9,2237,김용기,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B8%B0


In [100]:
sktest.shape

(9960, 24)

In [101]:
sktest_updated = sktest.merge(log,how="left",on="Name_Native")
sktest_updated.shape

(10178, 26)

In [102]:
# update URL_NamuWiki with log results
sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"URL_NamuWiki"] = sktest_updated.loc[sktest_updated.URL_NamuWiki.isnull() & sktest_updated.url.notnull(),"url"]

# confirm that changes have been made
sktest_updated.loc[sktest_updated.url.notnull(),["URL_NamuWiki","url"]]

,URL_NamuWiki,url
60,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD
679,https://namu.wiki/w/%EA%B9%80%EA%B2%BD%EC%97%B4,https://namu.wiki/w/%EA%B9%80%EA%B2%BD%EC%97%B4
1716,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B0%91,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B0%91
1717,False,False
1718,False,False
1719,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1720,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1721,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1722,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1723,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0


In [103]:
# change "False" strings to boolean False
sktest_updated.loc[sktest_updated.URL_NamuWiki=="False","URL_NamuWiki"]=False

In [104]:
sktest_updated[sktest_updated["index"].notnull()]

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki,index,url
60,강민국,姜旻局,19710303,남,43,시·도의회의원선거,20140604,Legislator,경상남도,진주시,...,28248.0,75.31,NaN,정당인,경남대학교 대학원 졸업(법학박사),(전)경상남도 도지사 비서실장,(전)경상남도 정무보좌역,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD,61.0,https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD
679,김경열,金庚烈,19510326,남,63,구·시·군의회의원선거,20140604,Legislator,전라남도,보성군,...,1622.0,24.11,NaN,농업,순천고등학교 졸업,(전)조성면민회 사무국장,(전)조성면 총무계장,https://namu.wiki/w/%EA%B9%80%EA%B2%BD%EC%97%B4,711.0,https://namu.wiki/w/%EA%B9%80%EA%B2%BD%EC%97%B4
1716,김용갑,金容甲,19520822,남,61,시·도의회의원선거,20140604,Legislator,전라남도,나주시,...,NaN,NaN,NaN,정치인,전남대학교 행정대학원 행정학과 졸업,(전)국회의원비서,(전)민주당나주화순지역위원회부위원장,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B0%91,2230.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B0%91
1717,김용경,金容京,19491005,남,64,구·시·군의회의원선거,20140604,Legislator,전라남도,나주시,...,2069.0,15.52,NaN,농업,호남원예고등학교 졸업,(전)민주당 나주시지구당 주민자치 위원장,(현)공법단체 국가유공자 월남전참전자회 전남지부나주시지회장,False,2231.0,False
1718,김용권,金龍權,19650320,남,49,시·도의회의원선거,20140604,Legislator,전라북도,군산시,...,NaN,NaN,NaN,자영업,원광보건대학 방사선과 졸업,(현)YMCA 이사,(현)군산박대협동조합 대표,False,2232.0,False
1719,김용규,金容圭,19650729,남,48,구·시·군의회의원선거,20140604,Legislator,광주광역시,북구,...,NaN,NaN,NaN,사회복지사,광주대학교 산업대학원 사회복지학과 졸업,(현) 대한나눔복지회 광주광역시지회장,(현) 문흥중앙초등학교 운영위원장,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,2233.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1720,김용규,金容圭,19650729,남,48,구·시·군의회의원선거,20140604,Legislator,광주광역시,북구,...,NaN,NaN,NaN,사회복지사,광주대학교 산업대학원 사회복지학과 졸업,(현) 대한나눔복지회 광주광역시지회장,(현) 문흥중앙초등학교 운영위원장,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,2234.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1721,김용규,金龍圭,19680310,남,46,구·시·군의회의원선거,20140604,Legislator,충청북도,청주시흥덕구,...,7821.0,28.56,NaN,대정화학 대표,청주대학교 경제학과 졸업,(전)청주시작은도서관협의회 회장,(현)글마루작은도서관 관장,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,2233.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1722,김용규,金龍圭,19680310,남,46,구·시·군의회의원선거,20140604,Legislator,충청북도,청주시흥덕구,...,7821.0,28.56,NaN,대정화학 대표,청주대학교 경제학과 졸업,(전)청주시작은도서관협의회 회장,(현)글마루작은도서관 관장,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C,2234.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%9C
1723,김용균,金容均,19420218,남,74,비례대표국회의원선거,20160413,Party List Legislator,전국,전국,...,NaN,NaN,NaN,변호사,미국 조지워싱턴대학교 법학박사(1976. 9. ~ 1978. 2.),(전)국회의원,(현)헌정회 감사,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0,2235.0,https://namu.wiki/w/%EA%B9%80%EC%9A%A9%EA%B7%A0


In [105]:
# drop log column (url)
sktest_updated.drop(columns=["index","url"],inplace=True)

In [106]:
# drop duplicate rows
sktest_updated.drop_duplicates().shape

(9960, 24)

In [107]:
# drop duplicate politicians
sktest_updated.sort_values(["Name_Native","Name_Chinese","Birthdate","ElectionDate"],inplace=True)
sktest_updated.drop_duplicates(["Name_Native","Name_Chinese","Birthdate"],keep="last",inplace=True)

In [108]:
sktest_updated.shape

(9960, 24)

In [109]:
# refresh and save updated sktest
sktest = sktest_updated
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [110]:
# delete log file
os.remove(fileout_path+logfile_name)

In [33]:
# DEPRECATED - assign URL_NamuWiki link based on files already scraped - I needed this before I created a log file

# import os
# filelist = os.listdir(fileout_path)
# test = sktest
# url_stem = "https://namu.wiki"
# test[100:200]
# scraped = [f.split("_")[2].split(".")[0].strip() for f in filelist]
# for s in scraped:
#     url = url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8"))
#     # print(url_stem+"/w/"+urllib.parse.quote(s.encode("utf-8")))
#     test.loc[test.Name_Native==s,"URL_NamuWiki"]=url
# test[100:200]